This notebook is code for studying the BGS data from the LSS Catalogs prior to running group finding. There are blocks to create the "Merged Files" that are inputs for the preprocessing and groupfinding routines in the GroupCatalog class.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import astropy.coordinates as coord
import astropy.units as u
import astropy.io.fits as fits
from astropy.table import Table,join,vstack,unique,QTable
import sys
from urllib.parse import urljoin
from scipy.ndimage import gaussian_filter

if './SelfCalGroupFinder/py/' not in sys.path:
    sys.path.append('./SelfCalGroupFinder/py/')
from pyutils import *
from dataloc import *
from photoz import *
from bgs_helpers import *
import groupcatalog as gc
from nnanalysis import *
from plotting import *

%load_ext autoreload
%autoreload 2

In [112]:
KEEP_PASSES = 1
APP_MAG_CUT = 19.5 # BGS BRIGHT, though 19.54 for some cameras. I don't know if FLUX_R has been corrected for this.
#APP_MAG_CUT = 20.0 # BGS FAINT, and in SV3 it is 20.3 instead
Z_MIN = 0.001
Z_MAX = 0.5
KEEP_ONLY_OBSERVED = True

# Building Merged Files

## Build SV3 Merged File
This requires Y3 Merged file to be built first, because we add in Y3 galaxies to supplement the NN catalog.

In [6]:
sv3_table = Table.read(BGS_SV3_ANY_FULL_FILE, format='fits')
sv3_table.keep_columns(['TARGETID', 'SPECTYPE', 'DEC', 'RA', 'Z_not4clus', 'NUMOBS', 'FLUX_R', 'FLUX_G', 'PROB_OBS', 'ZWARN', 'DELTACHI2', 'NTILE', 'TILES', 'TILEID', 'MASKBITS'])
sv3_table.rename_column('Z_not4clus', 'Z')

sv3_df = sv3_table.to_pandas()


In [ ]:
sv3_table = Table.read(BGS_SV3_ANY_FULL_FILE, format='fits')
print(f"Read {len(sv3_table)} galaxies from {BGS_SV3_ANY_FULL_FILE}")

# The lost galaxies will not have fastspecfit rows I think
sv3_table = add_fastspecfit_columns(sv3_table, 'sv3')

# Filter to needed columns only and save
sv3_table.keep_columns(['TARGETID', 'SPECTYPE', 'DEC', 'RA', 'Z_not4clus', 'NUMOBS', 'FLUX_R', 'FLUX_G', 'PROB_OBS', 'ZWARN', 'DELTACHI2', 'NTILE', 'TILES', 'TILEID', 'MASKBITS', 'DN4000', 'DN4000_MODEL', 'ABSMAG01_SDSS_G', 'ABSMAG01_SDSS_R'])
sv3_table.rename_column('Z_not4clus', 'Z')

add_mag_columns(sv3_table)
sv3_table = add_NTILE_MINE_to_table(sv3_table, "sv3")

sv3_table.write(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits', overwrite='True')

add_photz_columns(IAN_BGS_SV3_MERGED_NOY3_FILE, IAN_PHOT_Z_FILE_NOSPEC) # For SV3 Analysis we want to analyze pure photo-z's, so use the version without external spec-z

In [ ]:
# Now remove galaxies in the patches of SV3 that have poor overlap with Y3
sv3_table: Table = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')
print(len(sv3_table))

sv3_table['region'] = tile_to_region(sv3_table['NEAREST_TILEIDS'][:,0])
to_remove = np.isin(sv3_table['region'], sv3_poor_y3overlap)
sv3_table.remove_rows(to_remove)
print(len(sv3_table))

sv3_table.write(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits', overwrite='True')


In [ ]:
# Now add Y3 galaxies to supplement the NN catalog, especialy valuable at the edges of the regions
# They won't go into the main catalog because their NTILE_MINE is < 10
sv3_table: Table = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')
y3_table: Table = Table.read(IAN_BGS_Y3_MERGED_FILE_KIBO, format='fits')
#print(np.mean(y3_table['PROB_OBS']))
# Let's cut Y3 data down to targets somewhat close to SV3 regions
# No point in keeping rest and slowing down code
gals_coord = coord.SkyCoord(ra=y3_table['RA']*u.degree, dec=y3_table['DEC']*u.degree, frame='icrs')
close_array = gc.get_objects_near_sv3_regions(gals_coord, 2.5) # 2.5 deg radius is generously around the center of each SV3 region

y3_table = y3_table[close_array]

print(f"{len(y3_table)} galaxies will be added for SV3 NN catalog")
assert (y3_table['NTILE_MINE'] > 9).sum() == 0, "Y3 shouldn't add any galaxies that will go into the catalog"

# if the y3 table doesn't have prob_obs as is the case in JURA, add a prob_obs with 0.5 for everything
#if 'PROB_OBS' not in y3_table.columns:
#    print("PROB_OBS column missing from Y3")
#    y3_table.add_column(np.full(len(y3_table), 0.5), name="PROB_OBS")

# Find targets in Y3 that are already in SV3
y3_in_sv3 = np.isin(y3_table['TARGETID'], sv3_table['TARGETID'])
sv3_in_y3 = np.isin(sv3_table['TARGETID'], y3_table['TARGETID'])
assert y3_in_sv3.sum() == sv3_in_y3.sum()
print(f"Common targets between Y3 and SV3: {y3_in_sv3.sum()}; {y3_in_sv3.sum() / len(y3_table):.2%} of Y3 cut and {sv3_in_y3.sum() / len(sv3_table):.2%} of SV3")

In [ ]:
# Steal p_obs from SV3 for Y3 galaxies that are already in SV3
print(f"Stealing {y3_in_sv3.sum()} p_obs values from  Y3 galaxies for SV3")
sv3_table['PROB_OBS'][sv3_in_y3] = y3_table['PROB_OBS'][y3_in_sv3]
sv3_table['PROB_OBS'][~sv3_in_y3] = np.nan #0.689

In [ ]:
# Remove rows from y3_table that have TARGETID already in sv3_table
print(f"Removing {y3_in_sv3.sum()} (of {len(y3_table)}) Y3 galaxies that are already in SV3 catalog")
y3_table.remove_rows(y3_in_sv3)

#colname = 'NEAREST_TILEIDS'
#print(sv3_table[colname].shape)
#print(y3_table[colname].shape)

sv3_table.remove_column('TILES')
y3_table.remove_column('TILES')

combined = vstack([sv3_table, y3_table], join_type='outer')

# Ensure resultant data is what we want
print(sv3_table.columns)
print(y3_table.columns)
print(combined.columns)

In [13]:
combined.write(IAN_BGS_SV3_MERGED_FILE, format='fits', overwrite=True)

## Make Empty Photo-z Ledger file
 
See photoz.py for the code that populates this file.

In [ ]:
# INITIALIZE LIGHTWEIGHT DESI BGS PHOTO-Z TABLE
# Don't re-run! Will overwrite the file.
"""
desi_table = Table.read(IAN_BGS_Y3_MERGED_FILE, format='fits')
desi_table2 = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')

assert len(np.unique(desi_table['TARGETID'])) == len(desi_table), "There are duplicate TARGETIDs in the Y3 file"
assert len(np.unique(desi_table2['TARGETID'])) == len(desi_table2), "There are duplicate TARGETIDs in the SV3 file"

desi_table.keep_columns(['TARGETID', 'RA', 'DEC'])
desi_table2.keep_columns(['TARGETID', 'RA', 'DEC'])

desi_targets_table = vstack([desi_table, desi_table2], join_type='inner')
desi_targets_table = unique(desi_targets_table, 'TARGETID')
desi_targets_table['Z_LEGACY_BEST'] = NO_PHOTO_Z

# add columns for 'RELEASE', 'BRICKID', 'OBJID', 'REF_CAT', 'MATCH_DIST' with no values
desi_targets_table.add_column(np.zeros(len(desi_targets_table), dtype=int), name='RELEASE')
desi_targets_table.add_column(np.zeros(len(desi_targets_table), dtype=int), name='BRICKID')
desi_targets_table.add_column(np.zeros(len(desi_targets_table), dtype=int), name='OBJID')
desi_targets_table.add_column(np.zeros(len(desi_targets_table), dtype='S2'), name='REF_CAT')
desi_targets_table.add_column(np.full(len(desi_targets_table), 999999, dtype=float), name='MATCH_DIST')


desi_targets_table = desi_targets_table.to_pandas()
desi_targets_table.set_index('TARGETID', inplace=True)
pickle.dump(desi_targets_table, open(IAN_PHOT_Z_FILE, 'wb'))
"""


In [ ]:
# Read BGS_IMAGES_FOLDER + "terminal.txt" into an array of strings, 1 per line
f = open(BGS_IMAGES_FOLDER + "terminal.txt", "r")
lines = f.readlines()
f.close()

# Now filter it down to ones like this:
#Start processing brick #X
#Matched 0 out of Z
#start_lines = [line for line in lines if "Start processing brick" in line]
matched_lines = [line for line in lines if "Matched" in line]

#print(len(start_lines))
print(len(matched_lines))

# Extract X, Y, Z from each line 
#start_lines = [line.split()[3] for line in start_lines]    
matched_lines = [int(line.split()[1]) for line in matched_lines]

# Strip away # and convert to int
#start_lines = [int(line[1:]) for line in start_lines]

# The index is the brick number for matched_lines
# Find the brick numbers where the value is 0 and remember those indexes
zero_indexes = [i for i in range(len(matched_lines)) if matched_lines[i] == 0]

#pickle.dump(zero_indexes, open(BRICKS_TO_SKIP_S_FILE, 'wb'))

In [ ]:
#pickle.dump([], open(BRICKS_TO_SKIP_N_FILE, 'wb'))

## Build Y1 Merged File

In [ ]:
create_merged_file(BGS_ANY_FULL_FILE, IAN_BGS_MERGED_FILE, "1")

In [ ]:
# Make a file with just TARGETID and QUIESCENT columns to send to Joe for Clustering
tbl = Table.read(IAN_BGS_MERGED_FILE, format='fits') 
tbl.keep_columns(['TARGETID', 'QUIESCENT'])
tbl.write(BGS_Y1_FOLDER + "COLOR_LOOKUP_IRON.fits", format='fits', overwrite=True)

## Build Y3 Merged File

In [ ]:
build_sv3_clustering_randoms_files()

In [ ]:
build_sv3_full_randoms_files()

In [ ]:
create_merged_file(BGS_Y3_ANY_FULL_FILE, IAN_BGS_Y3_MERGED_FILE_KIBO, "3")

In [137]:
# Extra step
# Add columns to allow the possibility to make a Y3 catalog that is cut to SV3 regions.
table = Table.read(IAN_BGS_Y3_MERGED_FILE_KIBO, format='fits')
sv3tiles = read_tiles_Y3_sv3()

galaxies_df = table_to_df(table)

ntiles_inside, nearest_tile_ids = find_tiles_for_galaxies(sv3tiles, galaxies_df, 10)
if 'NTILE_MINE_SV3' in table.columns:
    table.remove_columns(['NTILE_MINE_SV3'])
if 'NEAREST_TILEIDS_SV3' in table.columns:
    table.remove_columns(['NEAREST_TILEIDS_SV3'])
table.add_column(ntiles_inside, name="NTILE_MINE_SV3")
table.add_column(nearest_tile_ids, name="NEAREST_TILEIDS_SV3")

table.write(IAN_BGS_Y3_MERGED_FILE_KIBO, format='fits', overwrite=True)

In [138]:
# Make a file with just TARGETID and QUIESCENT columns to send to Joe for Clustering
tbl = Table.read(IAN_BGS_Y3_MERGED_FILE_KIBO, format='fits') 
tbl.keep_columns(['TARGETID', 'QUIESCENT'])
tbl.write(BGS_Y3_FOLDER_KIBO + "COLOR_LOOKUP_KIBO.fits", format='fits', overwrite=True)

# Examine data in a Merged BGS File (SV3, Y1, Y3, whatever)

In [139]:
# Pick one
#table = Table.read(IAN_BGS_MERGED_FILE, format='fits')
#table = Table.read(IAN_BGS_SV3_MERGED_FILE, format='fits')
#table = Table.read(IAN_BGS_SV3_MERGED_NOY3_FILE, format='fits')
table = Table.read(IAN_BGS_Y3_MERGED_FILE_KIBO, format='fits')

## Cut to the galaxy data we actually need

In [ ]:
# TODO this gets easilly out of sync with the .py file that does the 'production' filtering

if np.ma.is_masked(table['Z']):
    print("Masked table")
    z_obs = table['Z'].data.data.astype("<f8")
    obj_type = table['SPECTYPE'].data.data
    unobserved = table['Z'].mask # the masked values are what is unobserved
    deltachi2 = table['DELTACHI2'].data.data  
else:
    print("Unmasked table")
    # SV3 version didn't do this
    z_obs = table['Z']
    obj_type = table['SPECTYPE']
    unobserved = table['Z'].astype("<i8") == 999999
    deltachi2 = table['DELTACHI2']

maskbits = table['MASKBITS'].data.data if np.ma.is_masked(table['MASKBITS']) else table['MASKBITS']
dec = table['DEC'].astype("<f8")
ra = table['RA'].astype("<f8")
z_phot = table['Z_PHOT'].astype("<f8")
target_id = table['TARGETID']
app_mag_r = get_app_mag(table['FLUX_R'])
app_mag_g = get_app_mag(table['FLUX_G'])
flux_r = table['FLUX_R'].astype("<f8")
flux_g = table['FLUX_G'].astype("<f8")
g_r_apparent = app_mag_g - app_mag_r
abs_mag_r = table['ABS_MAG_R']
abs_mag_g = table['ABS_MAG_G']
#sdss_g_r = table['ABSMAG_SDSS_G'] - table['ABSMAG_SDSS_R'] 
G_R_JM1 = table['ABSMAG01_SDSS_G'] - table['ABSMAG01_SDSS_R']
p_obs = table['PROB_OBS'] 
ntiles = table['NTILE'].astype("<i8")
#tiles = table['TILES']
tile_id = table['TILEID'] if 'TILEID' in table.columns else None
#numobs = table['NUMOBS']
#tile_locid = table['TILELOCID']
ntiles_mine = table['NTILE_MINE']
tileids = table['NEAREST_TILEIDS'][:,0].astype("<i8") # TODO there are 10 here, we want NTILES_MINE many...
#abs_mag_sdss = table['ABSMAG_SDSS_R']
dn4000 = table['DN4000_MODEL'].data.data
ref_cat = table['REF_CAT']
quiescent = table['QUIESCENT']


before_count = len(dec)
print(before_count, "objects in FITS file")

# TODO BUG Can we be mistaking STARS for GALAXIES?
# Make filter array (True/False values)
PASSES_REQUIRED = [1,2,3,4,10]

galaxy_observed_filter = obj_type == b'GALAXY'
app_mag_filter = app_mag_r < APP_MAG_CUT
redshift_filter = z_obs > Z_MIN
redshift_hi_filter = z_obs < Z_MAX
deltachi2_filter = deltachi2 > 40
#abs_mag_sdss_filter = abs_mag_sdss < 100
#observed_requirements = np.all([galaxy_observed_filter, app_mag_filter, redshift_filter, redshift_hi_filter, deltachi2_filter, abs_mag_sdss_filter], axis=0)
observed_requirements = np.all([galaxy_observed_filter, app_mag_filter, redshift_filter, redshift_hi_filter, deltachi2_filter], axis=0)

treat_as_unobserved = np.all([galaxy_observed_filter, app_mag_filter, np.invert(deltachi2_filter)], axis=0)

unobserved = np.all([app_mag_filter, np.logical_or(unobserved, treat_as_unobserved)], axis=0)
keep = np.all([np.logical_or(observed_requirements, unobserved)], axis=0)

print("\nWhole sample:")
print(f"There are {len(obj_type):,} objects in the entire sample, of which {np.sum(galaxy_observed_filter):,} are observed galaxies.") 

for n in PASSES_REQUIRED:
    n_pass_filter = ntiles_mine >= n
    n_pass_filter_old = ntiles >= n
    unobserved_n = np.all([n_pass_filter, unobserved], axis=0)
    observed_requirements_n = np.all([n_pass_filter, observed_requirements], axis=0)
    keepn = np.all([np.logical_or(observed_requirements_n, unobserved_n)], axis=0)

    print(f"\n{n}-pass analysis (NTILE_MINE):")
    print(f"There are {np.sum(observed_requirements_n):,} galaxies in the <{APP_MAG_CUT} mag sample that pass our quality checks.")
    print(f"There are {np.sum(unobserved_n):,} unobserved galaxies, including bad observed galaxies.")
    print(f"This {n}-pass catalog would have {np.sum(keepn):,} galaxies ({np.sum(unobserved_n) / np.sum(keepn) * 100:.2f}% lost).")

    # We've demonstratred this is definetely not what we want
    #unobserved_n_old = np.all([n_pass_filter_old, unobserved], axis=0)
    #observed_requirements_n_old = np.all([n_pass_filter_old, observed_requirements], axis=0)
    #keepn_old = np.all([np.logical_or(observed_requirements_n_old, unobserved_n_old)], axis=0)
    #print(f"\n{n}-pass analysis (NTILE):")
    #print(f"There are {np.sum(observed_requirements_n_old):,} galaxies in the bright (<{APP_MAG_CUT} mag) sample that pass our quality checks.")
    #print(f"There are {np.sum(unobserved_n_old):,} unobserved galaxies, including bad observed galaxies.")
    #print(f"This {n}-pass catalog would have {np.sum(keepn_old):,} galaxies ({np.sum(unobserved_n_old) / np.sum(keepn_old) * 100:.2f}% lost).")

# FOR PARTS BELOW SET WHAT YOU WANT TO KEEP!
if KEEP_ONLY_OBSERVED:
    keep = np.all([keep, ntiles_mine >= KEEP_PASSES, ~unobserved], axis=0)
else:
    keep = np.all([keep, ntiles_mine >= KEEP_PASSES], axis=0)

obj_type = obj_type[keep]
dec = dec[keep]
ra = ra[keep]
z_phot = z_phot[keep]
z_obs = z_obs[keep]
target_id = target_id[keep] 
flux_r = flux_r[keep]
app_mag_r = app_mag_r[keep]
app_mag_g = app_mag_g[keep]
g_r_apparent = g_r_apparent[keep]
p_obs = p_obs[keep]
unobserved = unobserved[keep]
deltachi2 = deltachi2[keep]
ntiles = ntiles[keep]
abs_mag_r = abs_mag_r[keep]
abs_mag_g = abs_mag_g[keep]
#tiles = tiles[keep]
#ztileid = ztileid[keep]
ntiles_mine = ntiles_mine[keep]
tileids = tileids[keep]
tile_id = tile_id[keep] if 'TILEID' in table.columns else None
#numobs = numobs[keep]
#tile_locid = tile_locid[keep]
#abs_mag_sdss = abs_mag_sdss[keep]
#sdss_g_r = sdss_g_r[keep]
G_R_JM1 = G_R_JM1[keep]
dn4000 = dn4000[keep]
ref_cat = ref_cat[keep]
maskbits = maskbits[keep]
indexes_not_assigned = np.flatnonzero(unobserved)
quiescent = quiescent[keep]

after_count = len(dec)

print(f"\nAfter all filters we have {after_count:,} of the original {before_count:,} rows.")

In [ ]:
# Make maps
two_pass_filter = ntiles_mine >= 2 
three_pass_filter = ntiles_mine >= 3 
four_pass_filter = ntiles_mine >= 4 

ra2 = ra[two_pass_filter]
dec2 = dec[two_pass_filter]
tileids2 =  tileids[two_pass_filter]
unobserved2 = unobserved[two_pass_filter]

ra3 = ra[three_pass_filter]
dec3 = dec[three_pass_filter]
tileids3 =  tileids[three_pass_filter]
unobserved3 = unobserved[three_pass_filter]

ra4 = ra[four_pass_filter]
dec4 = dec[four_pass_filter]
tileids4 =  tileids[four_pass_filter]
unobserved4 = unobserved[four_pass_filter]

one_pass_df = pd.DataFrame({'RA': ra, 'DEC': dec, 'Z_ASSIGNED_FLAG': unobserved, 'TILEID': tileids})
two_pass_df = pd.DataFrame({'RA': ra2, 'DEC': dec2, 'Z_ASSIGNED_FLAG': unobserved2, 'TILEID': tileids2})
three_pass_df = pd.DataFrame({'RA': ra3, 'DEC': dec3, 'Z_ASSIGNED_FLAG': unobserved3, 'TILEID': tileids3})
four_pass_df = pd.DataFrame({'RA': ra4, 'DEC': dec4, 'Z_ASSIGNED_FLAG': unobserved4, 'TILEID': tileids4})

#fig=make_map(ra, dec)
#ra_4 = ra[four_pass_filter]
#dec_4 = dec[four_pass_filter]
#print(f"Number of 4-pass galaxies: {len(ra_4)}, number of 3-pass galaxies: {len(ra)}")
#fig=make_map(ra_4, dec_4, fig=fig, alpha=0.1)

In [ ]:
plt.hist(ntiles_mine)

In [ ]:
tiles_BGS = read_tiles_Y1_main()
plot_positions(one_pass_df, three_pass_df, tiles_df=tiles_BGS, DEG_LONG=5, split=False)


In [ ]:
# See where missing photo-z's are
#idx_no_zphot = z_phot == NO_PHOTO_Z
#no_photoz_df = pd.DataFrame({'RA': ra[idx_no_zphot], 'DEC': dec[idx_no_zphot], 'Z_ASSIGNED_FLAG': unobserved[idx_no_zphot], 'TILEID': tileids[idx_no_zphot]})
#plot_positions(one_pass_df, no_photoz_df, tiles_df=tiles_BGS, DEG_LONG=3, split=False)

fig=make_map(ra3, dec3, dpi=300, alpha=0.02)
#ra_4 = ra[four_pass_filter]
#dec_4 = dec[four_pass_filter]
#print(f"Number of 4-pass galaxies: {len(ra_4)}, number of 3-pass galaxies: {len(ra)}")
#fig=make_map(ra[idx_no_zphot], dec[idx_no_zphot], fig=fig, alpha=0.1)
#fig=make_map(ra[ntiles_mine >= 10 ], dec[ntiles_mine >= 10 ], fig=fig, alpha=0.2)



In [ ]:
make_map(ra, dec, fig=fig, alpha=0.02)


### Make the random redshift hashtables

In [ ]:
# For comparison
with open(IAN_MXXL_LOST_APP_TO_Z_FILE, 'rb') as f:
    mxxl_app_mag_bins, mxxl_the_map = pickle.load(f)

In [ ]:
# This is how we did it for MXXL
app_mag_bins, the_map = build_app_mag_to_z_map(app_mag_r[~unobserved], z_obs[~unobserved])

In [ ]:
# New better way I think
app_mag_bins2, the_map2 = build_app_mag_to_z_map_2(app_mag_r[~unobserved], z_obs[~unobserved])

In [ ]:
app_mag_bins2

In [ ]:
for key in the_map2.keys():
    print(f"App mag bin {key} has {len(the_map2[key])} galaxies.")

In [ ]:
for i in range(100):
    m = app_mag_r[~unobserved][i]
    zp = z_phot[~unobserved][i]
    distribution = the_map2[np.digitize(m, app_mag_bins2)]
    percentiles = np.percentile(distribution, [5, 16, 50, 84, 95])
    mean = np.mean(distribution)
    std = np.std(distribution)
    # what sigma does zp lie at?
    sigma = (zp - mean) / std
    print(f"App mag: {m:.2f}, z_p: {zp:.4f}, sigma: {sigma:.2f}")

In [ ]:
# And now the photo-z infused way (there is a version 3 as well which fills empty/small bins with neighborin values)
app_mag_bins3, z_phot_bins, the_map3 = build_app_mag_to_z_map_4(app_mag_r[~unobserved].copy(), z_phot[~unobserved].copy(),  z_obs[~unobserved].copy())

In [ ]:
with np.printoptions(precision=4, linewidth=200):
    for k in the_map3.keys():
        print(f"Key {k} has {len(the_map3[k])} galaxies")
        #print(f"Key {k} has {len(the_map3[k])} galaxies: {np.percentile(the_map3[k], [5, 16, 50, 84, 95])}")

In [ ]:
bins = np.linspace(0, 0.5, 50)

#print(bins)
plt.figure(figsize=(10, 6))
trash=plt.hist(the_map2[33],bins=bins, color='green', density=True, histtype='step', label=app_mag_bins2[33], linestyle='dashed')
trash=plt.hist(the_map2[284],bins=bins, color='darkred', density=True, histtype='step', label=app_mag_bins2[284], linestyle='dashed')

trash=plt.hist(the_map3[39, 10], bins=bins, color=[0.0, 1.0, 0.0], density=True, histtype='step', label=f"z={z_phot_bins[10]:.3f} r={app_mag_bins3[39]:.2f}", linestyle='dotted')
trash=plt.hist(the_map3[30, 10], bins=bins, color=[0.1, 0.8, 0.0], density=True, histtype='step', label=f"z={z_phot_bins[10]:.3f} r={app_mag_bins3[30]:.2f}", linestyle='dotted')
trash=plt.hist(the_map3[22, 10], bins=bins, color=[0.2, 0.6, 0.0], density=True, histtype='step', label=f"z={z_phot_bins[10]:.3f} r={app_mag_bins3[22]:.2f}", linestyle='dotted')
trash=plt.hist(the_map3[15, 10], bins=bins, color=[0.3, 0.4, 0.0], density=True, histtype='step', label=f"z={z_phot_bins[10]:.3f} r={app_mag_bins3[15]:.2f}", linestyle='dotted')

trash=plt.hist(mxxl_the_map[29],bins=bins, color='green', density=True, histtype='step', label=mxxl_app_mag_bins[29])
#trash=plt.hist(app_mag_bins_read[50],bins=bins, color='orange', density=True, histtype='step', label=app_mag_bins_read[50], linestyle='dotted')
trash=plt.hist(mxxl_the_map[90],bins=bins, color='darkred', density=True, histtype='step', label=mxxl_app_mag_bins[90])
plt.legend()

plt.xlim(0, 0.5)



In [ ]:
with open(BGS_Y3_LOST_APP_TO_Z_FILE, 'wb') as f:
    pickle.dump((app_mag_bins2, the_map2), f)

In [ ]:
with open(BGS_Y3_LOST_APP_AND_ZPHOT_TO_Z_FILE, 'wb') as f:
    pickle.dump((app_mag_bins3, z_phot_bins, the_map3), f)

### Make SDSS BGS-Cut Variant

In [ ]:
fig=make_map(ra3, dec3)

sdss_df = pd.read_csv(SDSS_v2_DAT_FILE, header=None, sep=' ', names=['RA', 'DEC', 'Z', 'LOGLGAL', 'VMAX', 'color_flag', 'chi'])
sdss_gp_df = pd.read_csv(SDSS_v2_GALPROPS_FILE, delimiter=' ', header=None, names=('MAG_G', 'MAG_R', 'SIGMA_V', 'DN4000', 'CONCENTRATION', 'LOG_M_STAR', 'Z_ASSIGNED_FLAG'))

#fig=make_map(sdss_df['RA'].to_numpy(), sdss_df['DEC'].to_numpy(), alpha=0.3, fig=fig)
make_map(sdss_df.loc[sdss_gp_df.Z_ASSIGNED_FLAG == AssignedRedshiftFlag.NEIGHBOR_ONE.value, 'RA'].to_numpy(), sdss_df.loc[sdss_gp_df.Z_ASSIGNED_FLAG == AssignedRedshiftFlag.NEIGHBOR_ONE.value, 'DEC'].to_numpy(), alpha=0.5, dotsize=0.05, fig=fig)

In [ ]:
sra = sdss_df['RA'].to_numpy()    
ra_angles = coord.Angle(sra*u.degree)
ra_angles = ra_angles.wrap_at(180*u.degree)
sras = ra_angles.value
sdec = sdss_df['DEC'].to_numpy()
print(sra.min(), sra.max())
print(sdec.min(), sdec.max())

#stripe_to_cut = np.logical_and(sras < 90, sras > -90) # removes the 3 stripes
block_to_cut = np.logical_and(np.logical_and(sra > 130, sra < 195), np.logical_and(sdec > 15, sdec < 55))
block2_to_cut = np.logical_and(np.logical_and(sra > 210, sra < 225), np.logical_and(sdec > 15, sdec < 30))
rows_to_cut = np.logical_or(block2_to_cut, block_to_cut)

print(f"Cutting {np.sum(rows_to_cut)} rows from SDSS data")

sdss_cut = sdss_df.loc[~rows_to_cut]
make_map(sdss_cut['RA'].to_numpy(), sdss_cut['DEC'].to_numpy(), alpha=0.3, fig=fig)

In [ ]:
print(f"{spectroscopic_complete_percent(sdss_gp_df.Z_ASSIGNED_FLAG):.1%} of SDSS v2 galaxies have a redshift")
print(f"{spectroscopic_complete_percent(sdss_gp_df[~rows_to_cut].Z_ASSIGNED_FLAG):.1%} of SDSS BGS-Cut galaxies have a redshift")


In [ ]:
# Write this new catalog to a file
sdss_cut.to_csv(SDSS_BGSCUT_DAT_FILE, sep=' ', header=False, index=False)
sdss_gp_df.loc[~rows_to_cut].to_csv(SDSS_BGSCUT_GALPROPS_FILE, sep=' ', header=False, index=False)

In [ ]:
# Test reading the results
test1 = pd.read_csv(SDSS_BGSCUT_DAT_FILE, delimiter=' ', header=None, names=['RA', 'DEC', 'Z', 'LOGLGAL', 'VMAX', 'color_flag', 'chi'])
test2 = pd.read_csv(SDSS_BGSCUT_GALPROPS_FILE, delimiter=' ', header=None, names=('MAG_G', 'MAG_R', 'SIGMA_V', 'DN4000', 'CONCENTRATION', 'LOG_M_STAR', 'Z_ASSIGNED_FLAG'))
print(len(test1))
print(len(test2))

In [ ]:
# Known area of SDSS is .179, assuming constant density of galaxies we can estimate the area of the cut catalog
print(399434 / len(sra) * .179) 

# Compare with my crappy other estimation code
print(f"Orig area estimate: {estimate_frac_area(sra, sdec)}")
print(f"Cut area estimate: {estimate_frac_area(test1.RA.to_numpy(), test1['DEC'].to_numpy())}")

### Siena Galaxy Atlas Analysis (SGA)

In [ ]:
np.unique(ref_cat)

In [ ]:
# Understand how many galaxies are affected by Siena Galaxy Atlas (SGA) Masks
has_a_maskbit = maskbits != 0
idx_with_masks = np.flatnonzero(maskbits)
print(f"{np.sum(has_a_maskbit):,} galaxies ({np.sum(has_a_maskbit) / len(maskbits) * 100:.2f}%) have a maskbit set.")

unobserved_with_maskbits = np.logical_and(has_a_maskbit, unobserved)
print(f"{np.sum(unobserved_with_maskbits):,} galaxies ({np.sum(unobserved_with_maskbits) / len(maskbits) * 100:.2f}%) have a maskbit set and are unobserved.")

# See https://www.legacysurvey.org/dr9/bitmasks/
# https://github.com/legacysurvey/legacypipe/blob/master/py/legacypipe/bits.py
BITMASK_SGA = 0x1000 
sga_collision = (maskbits & BITMASK_SGA) != 0
print(f"{np.sum(sga_collision):,} galaxies ({np.sum(sga_collision) / len(maskbits) * 100:.2f}%) have a SGA collision.")

#sga_central = np.logical_or(ref_cat == b'L3', ref_cat == b'G2', ref_cat == b'GE')
sga_central = ref_cat == b'L3'
print(f"{np.sum(sga_central):,} galaxies ({np.sum(sga_central) / len(maskbits) * 100:.2f}%) are SGA centrals.")

to_remove = sga_collision & ~sga_central
print(f"{np.sum(to_remove):,} galaxies ({np.sum(to_remove) / len(maskbits) * 100:.2f}%) have a SGA collision and are not SGA centrals.")

# Examine color of these compared to the rest
g_r_sga = g_r_apparent[to_remove]
bins = np.linspace(-2, 2, 100)
plt.hist(g_r_sga, bins=bins, alpha=0.5, label='SGA Collision', density=True)
g_r_rest = g_r_apparent[np.logical_and(~unobserved, z_obs < 0.05)]
plt.hist(g_r_rest, bins=bins, alpha=0.5, label='Rest', density=True)
plt.legend()

to_remove_blue = to_remove & (g_r_apparent < 0.8)
print(f"{np.sum(to_remove_blue):,} galaxies ({np.sum(to_remove_blue) / len(maskbits) * 100:.2f}%) have a SGA collision, are not SGA centrals, and are blue enough to remove.")

df = pd.DataFrame({'RA': ra[to_remove_blue], 'DEC': dec[to_remove_blue]})
df.to_csv(OUTPUT_FOLDER + f'sga_collisions.csv', index=False)

df = pd.DataFrame({'RA': ra[to_remove & ~to_remove_blue], 'DEC': dec[to_remove & ~to_remove_blue]})
df.to_csv(OUTPUT_FOLDER + f'sga_collisions_not_blue_enough.csv', index=False)

# Inspecting these, I see that most are galaxies, not HII regions. Not sure if we want to remove

### Photo-Z vs Spec-Z Analysis 

In [ ]:
print(f"There are {(z_phot != NO_PHOTO_Z).sum():,} ({(z_phot != NO_PHOTO_Z).sum() / len(z_phot):.1%}) targets with phot-z")

good_idx = np.flatnonzero((z_phot != NO_PHOTO_Z) & ~unobserved) #& ~(np.isclose(z_phot, z_obs, atol=0.00025, rtol=0.000001)))
print(f"Amongst observed galaxies there are {len(good_idx):,} ({len(good_idx) / np.sum(~unobserved) * 100:.2f}%) galaxies with photo-z.")

unobserved_with_photz = np.flatnonzero((z_phot != NO_PHOTO_Z) & unobserved)
print(f"Amongst unobserved galaxies there are {len(unobserved_with_photz):,} ({len(unobserved_with_photz) / np.sum(unobserved) * 100:.2f}%) with photo-z.")

In [ ]:
delta_z = z_phot[good_idx] - z_obs[good_idx]
plt.hist(delta_z, bins=500, range=(-0.1, 0.1))
#plt.yscale("log")
plt.title("Photo-z Quality")
plt.ylabel("Count")
plt.xlabel("z_phot - z_spec")
plt.ylim(0, 20000)

# add bars for my z_thresh
plt.axvline(-SIM_Z_THRESH, color='red')
plt.axvline(SIM_Z_THRESH, color='red')

percentiles = np.percentile(delta_z, [16, 50, 84])
print(f"Median delta z: {percentiles[1]:.4f}, 16th percentile: {percentiles[0]:.4f}, 84th percentile: {percentiles[2]:.4f}")
percentiles = np.percentile(delta_z, [2.5, 97.5])
print(f"2.5th percentile: {percentiles[0]:.4f}, 97.5th percentile: {percentiles[1]:.4f}")
# add bars for the percentiles
#plt.axvline(percentiles[0], color='green')
#plt.axvline(percentiles[2], color='green')

# What % fall within 0.005 of the true redshift?
within_5_milli = np.abs(delta_z) < SIM_Z_THRESH
print(f"{np.sum(within_5_milli) / len(delta_z) * 100:.2f}% of galaxies have a photometric redshift within {SIM_Z_THRESH} of the spectroscopic redshift.")

# Now look only at quiescent galaxies less than 10^9 solar luminosities
# TODO 
#luminosity = abs_mag_r_to_log_solar_L(app_mag_to_abs_mag_k(app_mag_r, z_obs, g_r_apparent))


In [ ]:
from scipy.optimize import curve_fit
import numpy as np

import matplotlib.pyplot as plt

# Define Gaussian and Lorentzian functions
def gaussian(x, amp, mean, stddev):
    return amp * np.exp(-((x - mean) ** 2) / (2 * stddev ** 2))

def lorentzian(x, amp, mean, gamma, exp):
    return amp * gamma**2 / (np.power(np.abs(x - mean), exp) + gamma**2)

# Prepare data
abs_delta_z = delta_z
hist, bin_edges = np.histogram(abs_delta_z, bins=1000, range=(-0.2, 0.2), density=True)
bin_centers = (bin_edges[:-1] + bin_edges[1:]) / 2

# Fit Gaussian
popt_gauss, _ = curve_fit(gaussian, bin_centers, hist, p0=[1, 0, 0.01], maxfev=100000)
# Poor fit

# Fit Lorentzian
popt_lorentz, _ = curve_fit(lorentzian, bin_centers, hist, p0=[24, 0.0001, 0.0153, 2.0], maxfev=100000)
#Lorentzian fit parameters: amp=22.902018525656057, mean=8.151405674056218e-05, gamma=0.009227129158691942, exp=2.2857148452956757


# Plot histogram and fits
plt.figure(figsize=(10, 6))
plt.hist(abs_delta_z, bins=1000, range=(-0.2, 0.2), density=True, alpha=0.6, label='Data')
plt.plot(bin_centers, gaussian(bin_centers, *popt_gauss), label='Gaussian fit', color='red')
plt.plot(bin_centers, lorentzian(bin_centers, *popt_lorentz), label='~Lorentzian fit', color='green')
plt.xlabel('z_phot - z_spec')
plt.ylabel('Density')
plt.legend()
plt.title('Fitting delta_z with Gaussian and ~Lorentzian')
plt.show()

# Print fit parameters
print(f"Gaussian fit parameters: amp={popt_gauss[0]}, mean={popt_gauss[1]}, stddev={popt_gauss[2]}")
print(f"Lorentzian fit parameters: amp={popt_lorentz[0]}, mean={popt_lorentz[1]}, gamma={popt_lorentz[2]}, exp={popt_lorentz[3]}")

# SV3 Analysis

SV3 is composed of 20 regions where 10 or 11 exposures eacj were taken, almost completely on top of each other.  Our SV3 analysis takes the inner part of these patches (NTILE_MINE >= 10) of these regions as the data set.  

Then, we can eliminate 1 tile from each of these regions to make test sets in order to view our systematics as a function of NTILE_MINE. The order they are eliminated in matters; we need to go backwards in time.}

In [ ]:
# Make a DataFrame filtered down to the galaxies we want to keep
sv3_merged_table = Table.read(IAN_BGS_SV3_MERGED_FILE, format='fits')
sv3_merged_table.remove_column('NEAREST_TILEIDS')
sv3_df = sv3_merged_table.to_pandas()
print(len(sv3_df))
sv3_df['APP_MAG_R'] = get_app_mag(sv3_df['FLUX_R'])
unobserved = sv3_merged_table['Z'].astype("<i8") == 999999
galaxy_observed_filter = sv3_df['SPECTYPE'] == b'GALAXY'
redshift_filter = sv3_df['Z'] > Z_MIN
redshift_hi_filter = sv3_df['Z'] < Z_MAX
deltachi2_filter = sv3_df['DELTACHI2'] > 40
app_mag_filter = sv3_df['APP_MAG_R'] < 20.3
observed_requirements = np.all([galaxy_observed_filter, app_mag_filter, redshift_filter, redshift_hi_filter, deltachi2_filter], axis=0)
treat_as_unobserved = np.all([galaxy_observed_filter, app_mag_filter, np.invert(deltachi2_filter)], axis=0)

unobserved = np.all([app_mag_filter, np.logical_or(unobserved, treat_as_unobserved)], axis=0)
sv3_df['OBSERVED'] = np.invert(unobserved)
keep = np.all([np.logical_or(observed_requirements, unobserved)], axis=0)
keep = np.all([keep, sv3_df['NTILE_MINE'] >= 10], axis=0)

sv3_df = sv3_df.loc[keep] 
sv3_df.reset_index(drop=True, inplace=True)
print(len(sv3_df))

# Initialize new columns for observed as function of N pass
for i in range(0, 12):
    sv3_df[f'OBSERVED_{i}'] = sv3_df['OBSERVED']

for FAINT in [False, True]:

    if not FAINT:
        mag_filter = sv3_df['APP_MAG_R'] < 19.5
    else:
        mag_filter = sv3_df['APP_MAG_R'] > 19.5
        
    print(f"{len(sv3_df[mag_filter]) / 138.192} galaxies per sq degree")

    for patch_number in range(len(sv3_regions_sorted)):
        tilelist = sv3_regions_sorted[patch_number]
        #print(f'Patch {patch_number} - TILE IDs: {tilelist}')
        
        row_selector = np.logical_and(sv3_df['TILEID'].isin(tilelist), mag_filter)

        #one_patch_df = sv3_df[sv3_df['TILEID'].isin(tilelist)]
        #print(f"{len(one_patch_df)} galaxies, {np.sum(one_patch_df['OBSERVED']) / len(one_patch_df) :.1%} of the targets are observed")
        #one_patch_df[f'OBSERVED_{len(tilelist)}'] = one_patch_df['OBSERVED']
        
        #print ("Remove tiles in reverse TILEID order:")
        for i in np.flip(np.arange(0, len(tilelist))):
            tileid = tilelist[i]
            observed_by_this_tile = sv3_df.loc[row_selector, 'TILEID'] == tileid
            #print(f'{np.sum(observed_by_this_tile)} galaxies were observed by tile {tileid} ({i+1}/{len(tilelist)})')
            prev = sv3_df.loc[row_selector, f'OBSERVED_{i+1}']
            sv3_df.loc[row_selector, f'OBSERVED_{i}'] = np.where(observed_by_this_tile, False, prev)
            
        #for i in np.flip(np.arange(0, len(tilelist)+1)):
        #    if FAINT:
        #        totals_observed_faint[i] += np.sum(sv3_df.loc[row_selector, f'OBSERVED_{i}'])
        #        totals_all_faint[i] += len(sv3_df.loc[row_selector])
        #    else:
        #        totals_observed_bright[i] += np.sum(sv3_df.loc[row_selector, f'OBSERVED_{i}'])
        #        totals_all_bright[i] += len(sv3_df.loc[row_selector])

            #print(f"{np.sum(one_patch_df[f'OBSERVED_{i}']) / len(one_patch_df) :.1%} of the targets are observed with {i} passes")
                
    #for i in range(1, 12):
    #    if FAINT:
    #        print(f"{totals_observed_faint[i]:,} ({totals_observed_faint[i] / totals_all_faint[i]:.1%}) faint galaxies are observed with {i} passes")
    #    else: 
    #        print(f"{totals_observed_bright[i]:,} ({totals_observed_bright[i] / totals_all_bright[i]:.1%}) bright galaxies are observed with {i} passes")


In [ ]:
observed_faint = np.zeros(12, dtype=int)
observed_bright = np.zeros(12, dtype=int)

total_faint = np.sum(sv3_df['APP_MAG_R'] > 19.5)
total_bright = np.sum(sv3_df['APP_MAG_R'] < 19.5)

for i in range(0, 12):
    observed_faint[i] = np.sum(sv3_df.loc[sv3_df['APP_MAG_R'] > 19.5, f'OBSERVED_{i}'])
    observed_bright[i] = np.sum(sv3_df.loc[sv3_df['APP_MAG_R'] < 19.5, f'OBSERVED_{i}'])


plt.plot(observed_bright / total_bright, color='b', label="BGS BRIGHT ME")
plt.plot(observed_faint / total_faint, color='orange', label="BGS FAINT ME")
plt.plot([1,2,3,4], [.29, .52, 0.68, .81], '--', color='b', label="BGS BRIGHT PAPER")
plt.plot([1,2,3,4], [.15, .32, 0.47, .62], '--', color='orange', label="BGS FAINT PAPER")
plt.xlabel("Number of passes")
plt.ylabel("Fraction of targets observed")
plt.title("SV3 BGS Completeness")
plt.xticks(np.arange(0, 12))
plt.legend()

In [ ]:
print(sv3_df.loc[sv3_df['APP_MAG_R'] < 19.5, 'OBSERVED'].count() - sv3_df.loc[sv3_df['APP_MAG_R'] < 19.5, 'OBSERVED'].sum())
print(sv3_df.loc[sv3_df['APP_MAG_R'] < 19.5, 'OBSERVED'].count())

The above numbers do not seem to track with the Y1 data. In Y1 no region has more than 4 passes so how do I have a fiber incompleteness better than the above number?

Also Figure 17 of https://iopscience.iop.org/article/10.3847/1538-3881/accff8/pdf disagrees with my above analysis. So what is above is wrong.

# Nearest Neighbor Analysis (for SV3 Merged File)

Have a nearest neighbor catalog that is all the actually observed galaxies in SV3 with my Y3 supplement. 

Can run analysis on entire sample or a subset of 'pretend-to-be unobserved' galaxies (~obs_7p).

### Create bin files

In [ ]:
# Calculate the percentiles for 10 equal bins
percentiles = np.linspace(0, 100, 16)
bin_edges = np.percentile(z_obs, percentiles)

with np.printoptions(precision=3, linewidth=200):
    print(f"Even number z bin: {np.array(bin_edges[1:])}")
    print(f"The Z Bins we use: {Z_BINS}")

In [ ]:
# MAKE SURE that above KEEP_PASSES is set to 1 and you filter out unobserved galaxies as we need a source of 'truth'
assert KEEP_PASSES == 1 and np.sum(unobserved) == 0
assert Z_MIN == 0.001 and Z_MAX == 0.5

gmr = app_mag_g - app_mag_r
Rk = app_mag_to_abs_mag_k(app_mag_r, z_obs, gmr, band='r')
Gk = app_mag_to_abs_mag_k(app_mag_g, z_obs, gmr, band='g')
quiescent = is_quiescent_BGS_gmr(None, Gk-Rk)

# We will only consider the observed galaxies in SV3 as we need a source of truth for the analysis.
# This is ~98% so this analysis should be representative.
# As usual for SV3, only want galaxies that were fully covered by the rosetting pattern. (in_10p_zone)
# We then pretend to have not observed ~20% of galaxies, as is the case in the main survey. (obs_7p)
# TODO BUG Ashley says my method is wrong for doing this.
in_10p_zone = ntiles_mine >= 10
obs_7p = ~gc.drop_SV3_passes(3, tile_id, unobserved)

print(np.sum(quiescent))
print(np.sum(in_10p_zone))
print(np.sum(obs_7p))

In [ ]:
tile_id.mask.sum() # BUG prevents using LOST_GALAXIES_ONLY=True. Is it because of Y3 supplement has no TILE_ID?

In [ ]:
###
### Create NEIGHBOR_ANALYSIS_SV3_BINS_FILE
###
p_obs = np.where(np.isnan(p_obs), 0.689, p_obs) # Fill in missing p_obs with 0.689, the Y3 mean
newobj = NNAnalyzer_cic.from_data(dec, ra, z_obs, app_mag_r, Rk, g_r_apparent, quiescent, obs_7p, p_obs)
#newobj.set_row_locator( np.logical_and(obs_10p, app_mag_r < 19.5) ) # 10p inner regions and BRIGHT only
newobj.set_row_locator(in_10p_zone)
newobj.find_nn_properties(LOST_GALAXIES_ONLY=False) 
newobj.make_bins()
newobj.save(NEIGHBOR_ANALYSIS_SV3_BINS_FILE)

print(np.sum(newobj.all_ang_bincounts))
print(np.sum(newobj.all_sim_z_bincounts))

In [ ]:
# Gaussian smoothing version
loaded_nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_FILE)

# Apply Gaussian filter
loaded_nna.apply_gaussian_smoothing(0.75)

# Save extra smoothed one
loaded_nna.save(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE)

### Look at bin results

In [ ]:
orig_nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_FILE)
smoothed_nna = NNAnalyzer_cic.from_results_file(NEIGHBOR_ANALYSIS_SV3_BINS_SMOOTHED_FILE)


In [ ]:
orig_nna.plot_angdist_appmag_per_zbin_cc(z_bin_numbers_to_plot=[2,4,6,8])

In [ ]:
smoothed_nna.plot_angdist_appmag_per_zbin_cc(z_bin_numbers_to_plot=[2,4,6,8])

In [ ]:
# These look OK 
orig_nna.plot_angdist_appmag_per_zbin_cc()

In [ ]:
print(np.shape(smoothed_nna.all_ang_bincounts))
print(np.prod(np.shape(smoothed_nna.all_ang_bincounts)))
print(np.median(smoothed_nna.all_ang_bincounts))
print(np.mean(smoothed_nna.all_ang_bincounts))

In [ ]:
# These make P_obs seem useles...
smoothed_nna.plot_angdist_pobs_per_zbin_cc(t_c=[1], nn_c=[0], z_bin_numbers_to_plot=[1,3,5,7])

In [ ]:
smoothed_nna.plot_angdist_pobs_per_zbin_cc(t_c=[1], nn_c=[1], z_bin_numbers_to_plot=[1,4,7,10])

In [ ]:
# TODO de-dupe with pre_process_BGS(...)

# Examine photo-z NN relation
NUM_NEIGHBORS = 30
unobs_7p = ~obs_7p
bright = app_mag_r[in_10p_zone] < 19.5
use = bright & unobs_7p
neighbor_indexes = np.zeros(shape=(NUM_NEIGHBORS, use.sum()), dtype=np.int32) # indexes point to CATALOG locations
ang_distances = np.zeros(shape=(NUM_NEIGHBORS, use.sum()))

catalog = coord.SkyCoord(ra=ra[in_10p_zone][obs_7p]*u.degree, dec=dec[in_10p_zone][obs_7p]*u.degree, frame='icrs')

print(f"Finding nearest {NUM_NEIGHBORS} neighbors... ", end='\r')   
for n in range(0, NUM_NEIGHBORS):
    to_match = coord.SkyCoord(ra=ra[in_10p_zone][use]*u.degree, dec=dec[in_10p_zone][use]*u.degree, frame='icrs')
    idx, d2d, d3d = coord.match_coordinates_sky(to_match, catalog, nthneighbor=n+1, storekdtree='sv3')
    neighbor_indexes[n] = idx
    ang_distances[n] = d2d.to(u.arcsec).value
print(f"Finding nearest {NUM_NEIGHBORS} neighbors... done!")   


In [ ]:
ztruth = z_obs[in_10p_zone][use]
# fill zphot_fake with ztruth + a random gaussian draw around 0.0 with sigma of 0.01
zphot = z_phot[in_10p_zone][use]


In [ ]:
one_pass_df.Z_ASSIGNED_FLAG = ~obs_7p
plot_positions(one_pass_df, tiles_df=None, DEG_LONG=2.5, split=True, ra_min=one_pass_df.RA[0], dec_min=one_pass_df['DEC'][0])

In [ ]:
# Test of how gaussian filter works

# Create a sample 2D array with zeros and a few ones
array = np.zeros((30, 30))
array[0, 0] = 1
array[1, 0] = 1
array[5, 5] = 1
array[8, 5] = 1
array[20, 20] = 3

# Apply Gaussian filter
sigma = 0.75  # Standard deviation for Gaussian kernel
smoothed_array = gaussian_filter(array, sigma=sigma, axes=[0,1])

assert np.isclose(np.sum(array), np.sum(smoothed_array))

# Plot the original and smoothed arrays
fig, ax = plt.subplots(1, 2, figsize=(12, 6))
ax[0].imshow(array, cmap='gray')
ax[0].set_title('Original Array')
ax[1].imshow(smoothed_array, cmap='gray')
ax[1].set_title('Smoothed Array')
plt.show()

### Initial look at Photo-z only matching to neighbors

In [ ]:
paramspace = np.arange(0.001, 0.05, 0.001)
                       
cumulative_percent_correct_by_n_zp_firstonly = np.zeros((NUM_NEIGHBORS, len(paramspace)) , dtype=float)

i = 0
for PHOTOZ_MATCHING_THRESHOLD in paramspace:
    
    this_neighbor_correct = np.zeros(shape=(NUM_NEIGHBORS, len(ztruth)), dtype=bool)
    z_phot_neighbor_match = np.zeros(shape=(NUM_NEIGHBORS, len(ztruth)), dtype=bool)
    cumulative_percent_z_phot_neighbor_match = []
    z_phot_first_neighbor_match_idx = np.ones(len(ztruth), dtype=int) * 999 # sentinal value for no match
    z_phot_match_correct = np.zeros(shape=(NUM_NEIGHBORS, len(ztruth)), dtype=bool)
    delta_z = np.zeros(shape=(NUM_NEIGHBORS, len(ztruth)), dtype=float)

    percent_correct_by_n = []
    percent_correct_by_n_zp = []
    cumulative_percent_correct_by_n = []
    cumulative_percent_correct_by_n_zp = []
    cumulative_percent_correct_by_n_zp_closestonly = []
    first_matched_but_incorrect = []

    for n in range(0, NUM_NEIGHBORS):

        z_neighbor = z_obs[in_10p_zone][obs_7p][neighbor_indexes[n]]
        this_neighbor_correct[n] = close_enough(ztruth, z_neighbor)
        percent_correct_by_n.append(this_neighbor_correct[n].sum() / len(ztruth))
        any_neighbor_correct = this_neighbor_correct[0:n+1].max(axis=0) # max will be True if any neighbor is True
        cumulative_percent_correct_by_n.append(any_neighbor_correct.sum() / len(ztruth))

        z_phot_neighbor_match[n] = close_enough(zphot, z_neighbor, threshold=PHOTOZ_MATCHING_THRESHOLD)
        any_z_phot_match = z_phot_neighbor_match[0:n+1].max(axis=0) # will be True if z_phot matches a neighbor
        cumulative_percent_z_phot_neighbor_match.append(any_z_phot_match.sum() / len(ztruth))
        z_phot_match_correct[n] = z_phot_neighbor_match[n] & this_neighbor_correct[n]
        any_neighbor_zp_correct = z_phot_match_correct[0:n+1].max(axis=0) # will be True if z_phot matches a neighbor
        percent_correct_by_n_zp.append(z_phot_match_correct[n].sum() / len(ztruth))
        cumulative_percent_correct_by_n_zp.append(any_neighbor_zp_correct.sum() / len(ztruth))


        # Now only consider the closest neighbor, by photo-z matching
        delta_z[n] = zphot - z_neighbor
        best_match_idx = np.argmin(delta_z[0:n+1], axis=0, keepdims=True)
        closest_delta_z_correct = np.take_along_axis(z_phot_match_correct, best_match_idx, axis=0)[0] # will be True bset zphot match is correct
        cumulative_percent_correct_by_n_zp_closestonly.append(closest_delta_z_correct.sum() / len(ztruth))

        # Now only consider the first neighbor in order of angular distance
        z_phot_first_neighbor_match_idx = np.minimum(z_phot_first_neighbor_match_idx, np.where(z_phot_neighbor_match[n],n,999))
        has_match = z_phot_first_neighbor_match_idx != 999

        # if z_phot_first_neighbor_match_idx is 999 for  row, first_correct will be False
        # if z_phot_first_neighbor_match_idx is an index for row, first_correct will be z_phot_match_correct value for that row at that index
        first_correct = np.repeat(False, len(ztruth))
        first_correct[has_match] = z_phot_match_correct[z_phot_first_neighbor_match_idx[has_match], np.arange(len(ztruth))[has_match]]
        cumulative_percent_correct_by_n_zp_firstonly[n,i] = (first_correct.sum() / len(ztruth))

        first_matched_but_incorrect.append(cumulative_percent_z_phot_neighbor_match[n] - cumulative_percent_correct_by_n_zp_firstonly[n,i])

    if i == 26:
        plt.figure(figsize=(10, 5))
        plt.plot(range(1, NUM_NEIGHBORS+1), percent_correct_by_n, label="This Neighbor is correct z")
        plt.plot(range(1, NUM_NEIGHBORS+1), percent_correct_by_n_zp, label="This Neighbor photo-z matched and correct (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n, label="Any Neighbor has correct z")
        plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp, label="Any neighbor photo-z matched and correct (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_closestonly, label="Minimum delta-z matched neighbor correct (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,i], label=f"First photo-z matched neighbor correct (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_z_phot_neighbor_match, label="Any Neighbor photo-z matched (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.ylabel("Fraction")
        plt.xlabel("Nth Nearest Neighbor")
        plt.xticks(np.arange(1, NUM_NEIGHBORS+1))
        plt.ylim(0, 0.8)
        #plt.axhline(y=0.0125, color='r', linestyle='--', label="Random Chance")
        plt.legend()
        
        plt.figure(figsize=(10, 5))
        # stacked bar chart of 
        plt.bar(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,i], label=f"First photo-z matched neighbor correct (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})")
        plt.bar(range(1, NUM_NEIGHBORS+1), first_matched_but_incorrect, label=f"First photo-z matched neighbor incorrect (thresh={PHOTOZ_MATCHING_THRESHOLD:.3})", bottom = cumulative_percent_correct_by_n_zp_firstonly[:,i])

        plt.ylabel("Fraction")
        plt.xlabel("Nth Nearest Neighbor")
        plt.xticks(np.arange(1, NUM_NEIGHBORS+1))
        plt.ylim(0, 1.0)
        plt.legend()
    
    i = i + 1





In [ ]:
# Find the maximum point
best = np.max(cumulative_percent_correct_by_n_zp_firstonly)
best_idx = np.unravel_index(np.argmax(cumulative_percent_correct_by_n_zp_firstonly), cumulative_percent_correct_by_n_zp_firstonly.shape)
print(cumulative_percent_correct_by_n_zp_firstonly.shape)
print(best_idx)
print(f"Best photo-z match threshold: {paramspace[best_idx[1]]:.3f}, Neighbors={best_idx[0]+1}, {best:.2%}")


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,4],        label=f"thresh={paramspace[4]:.3})", color=[0.1, 0.8, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,9],        label=f"thresh={paramspace[9]:.3})", color=[0.2, 0.7, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,19],       label=f"thresh={paramspace[19]:.3})", color=[0.3, 0.6, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,26], '--', label=f"thresh={paramspace[26]:.3})", color=[0.4, 0.5, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,39],       label=f"thresh={paramspace[39]:.3})", color=[0.5, 0.4, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,49],       label=f"thresh={paramspace[49]:.3})", color=[0.6, 0.3, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,59],       label=f"thresh={paramspace[59]:.3})", color=[0.7, 0.2, 0])
plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,99],       label=f"thresh={paramspace[99]:.3})", color=[0.8, 0.1, 0])
plt.ylabel("Fraction Correct")
plt.xlabel("Nth Nearest Neighbor")
plt.xticks(np.arange(1, NUM_NEIGHBORS+1))
plt.ylim(0, 0.45)
#plt.axhline(y=0.0125, color='r', linestyle='--', label="Random Chance")
plt.legend()

In [ ]:
# Make a figure showing the best photo-z match for each galaxy
plt.figure(figsize=(10, 5))
for i in range(0, len(paramspace)):
    color = [i/(len(paramspace)+1),0.5,0]
    plt.plot(range(1, NUM_NEIGHBORS+1), cumulative_percent_correct_by_n_zp_firstonly[:,i], label=f"Photo-z Match Threshold {paramspace[i]:.3f}", color=color)
plt.ylabel("Fraction Correct")
plt.xlabel("Nth Nearest Neighbor")
plt.xticks(np.arange(1, NUM_NEIGHBORS+1))
plt.ylim(0, 0.8)

# Color Analysis

Lesson from this analysis: the BGS data, workign with my 0.1^G-R with GAMA k-corrections, does not distribute a per logLgal bin G-R; the global 0.76 split seems to work for all bins.

In [141]:
G = app_mag_to_abs_mag(app_mag_g, z_obs)
R = app_mag_to_abs_mag(app_mag_r, z_obs)

G_R = G - R

In [142]:
# It doesn't matter if you use g_r_apparent or G_R as the difference between the is the same!

Gk = k_correct_bgs(G, z_obs, g_r_apparent, band='g')
Rk = k_correct_bgs(R, z_obs, g_r_apparent, band='r')
G_R_k_BGS1 = Gk - Rk

Gk_GAMA = k_correct_gama(G, z_obs, g_r_apparent, band='g')
Rk_GAMA = k_correct_gama(R, z_obs, g_r_apparent, band='r')
G_R_k_GAMA = Gk_GAMA - Rk_GAMA

Gk_BGS2 = k_correct_bgs_v2(G, z_obs, g_r_apparent, band='g')
Rk_BGS2 = k_correct_bgs_v2(R, z_obs, g_r_apparent, band='r')
G_R_k_BGS2 = Gk_BGS2 - Rk_BGS2

In [ ]:
# Comparison of g-r computed a few ways
bins = np.linspace(0, 2.0, 200)

plt.figure()
#junk=plt.hist(g_r_apparent, bins=bins, label="g-r", histtype='step', density=True)
#junk=plt.hist(sdss_g_r, bins=bins, label='From LSS Pipeline (JM?)', histtype='step', density=True)
junk=plt.hist(G_R_JM1, bins=bins, label="0.1^(G-R) JM", histtype='step', density=True)
#junk=plt.hist(G_R, bins=bins, label="G-R", histtype='step', density=True)
junk=plt.hist(G_R_k_BGS1, bins=bins, label="0.1^(G-R) BGS poly v1", histtype='step', density=True)
junk=plt.hist(G_R_k_GAMA, bins=bins, label="0.1^(G-R) GAMA poly", histtype='step', density=True)
junk=plt.hist(G_R_k_BGS2, bins=bins, label="0.1^(G-R) BGS poly v2", histtype='step', density=True)
plt.xlabel("g-r")
plt.ylabel("Count")
plt.legend()
plt.xlim(0.2, 1.3)
plt.title("Comparison of g-r computed a few ways")
plt.tight_layout()
plt.ylim(0,3.5)

In [ ]:
# Can see global GLOBAL_RED_COLOR_CUT=0.76 here
junk=plt.hist(G_R_k_GAMA, bins=300, alpha=0.5, label="0.1^(G-R) GAMA-style")
junk=plt.hist(G_R_JM1, bins=300, alpha=0.5, label="0.1^(G-R) JM")
plt.legend()
plt.xlim(0.5, 1.0)

In [ ]:
print(is_quiescent_lost_gal_guess(g_r_apparent).sum() / len(g_r_apparent))
assert len(G_R_k_GAMA) == len(g_r_apparent)
print(is_quiescent_BGS_gmr(None, G_R_k_GAMA).sum() / len(G_R_k_GAMA))

In [ ]:
from pyutils import *
print(BGS_LOGLGAL_BINS)
print(BINWISE_RED_COLOR_CUT)

In [ ]:
is_quiescent_BGS_gmr(np.array([5.8, 9.0, 14.5]), np.array([0.5, 0.9, 0.9]))

In [149]:
# Get logLgal bins
log_L_gal = abs_mag_r_to_log_solar_L(Rk) 

In [ ]:
# define 8 bins that have equal number of galaxies
log_L_gal_bins = np.percentile(log_L_gal, np.linspace(0, 100, 9))
print(log_L_gal_bins)

In [151]:
logLgal_bin_idx = np.digitize(log_L_gal, BGS_LOGLGAL_BINS)
# 0 is less than the lowest, len(BGS_LOGLGAL_BINS) is greater than the highest entry in BGS_LOGLGAL_BINS


In [ ]:
print(np.min(log_L_gal))
print(np.max(log_L_gal))
print(np.min(logLgal_bin_idx))
print(np.max(logLgal_bin_idx))
plt.hist(log_L_gal, bins=BGS_LOGLGAL_BINS, align='mid')
#plt.yscale('log')

In [ ]:
# Make a plot of G_R_k in each logLgal bin
for i in range(0, len(BGS_LOGLGAL_BINS)+1):
    galaxy_idx_for_this_bin = logLgal_bin_idx == i

    plt.figure(dpi=80, figsize=(5, 3))
    junk=plt.hist(G_R_k_GAMA[galaxy_idx_for_this_bin], bins=np.arange(0,1.3,0.02), label=f"0.1^(G-R) Bin {i}", align='mid')
    plt.legend()
    plt.xlim(0.4, 1.2)
    plt.xticks(np.arange(0.4, 1.2, 0.04))

## Dn4000 Comparison (BGS, SDSS)

In [154]:
sdss = pd.read_csv(SDSS_v1_DAT_FILE, delimiter=' ', names=('RA', 'DEC', 'Z', 'LOGLGAL', 'VMAX', 'QUIESCENT', 'chi'), index_col=False)
sdss_galprops = pd.read_csv(SDSS_v1_1_GALPROPS_FILE, delimiter=' ', names=('MAG_G', 'MAG_R', 'SIGMA_V', 'DN4000', 'CONCENTRATION', 'LOG_M_STAR', 'Z_ASSIGNED_FLAG'))
sdss = pd.merge(sdss, sdss_galprops, left_index=True, right_index=True)

In [ ]:
plt.hist(dn4000, bins=np.linspace(-0.5, 5.0, 100), alpha=0.6, label="BGS", density=True, histtype='step')    
plt.hist(sdss.DN4000, bins=np.linspace(-0.5, 5.0, 100), alpha=0.8, label="SDSS", density=True, histtype='step')
plt.yscale('log')
plt.legend()
plt.xlabel('DN4000')
plt.ylabel('Count')

In [ ]:
plt.hist(dn4000, bins=np.linspace(-0.5, 5.0, 100), alpha=0.6, label="BGS")
plt.hist(sdss.DN4000, bins=np.linspace(-0.5, 5.0, 100), alpha=0.8, label="SDSS")
plt.legend()
plt.xlabel('DN4000')
plt.ylabel('Count')
plt.xlim(0.9,2.5)

In [157]:
sdss_catalog = coord.SkyCoord(ra=sdss.RA.to_numpy()*u.degree, dec=sdss['DEC'].to_numpy()*u.degree, frame='icrs')
BGS_catalog = coord.SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')

neighbor_indexes, d2d, d3d = coord.match_coordinates_sky(BGS_catalog, sdss_catalog, storekdtree='sdss')
ang_distances = d2d.to(u.arcsec).value

match_found_filter = np.logical_and(ang_distances < 1.0, ~np.isnan(dn4000))
bgs_matches = dn4000[match_found_filter]
sdss_indexes = neighbor_indexes[match_found_filter]
sdss_matches = sdss.iloc[sdss_indexes].DN4000.to_numpy()

In [ ]:
print(f"{len(bgs_matches)} matches found (with Dn4000 available)")
print(f"{np.isclose(bgs_matches, sdss_matches, atol=0.05).sum() / len(bgs_matches)} of the matches are within 0.05 of each other.")
print(f"{np.isclose(bgs_matches, sdss_matches, atol=0.1).sum() / len(bgs_matches)} of the matches are within 0.1 of each other.")
print(f"{np.isclose(bgs_matches, sdss_matches, atol=0.2).sum() / len(bgs_matches)} of the matches are within 0.2 of each other.")
print(f"{np.isclose(bgs_matches, sdss_matches, atol=0.3).sum() / len(bgs_matches)} of the matches are within 0.3 of each other.")


In [ ]:
import numpy as np

difference = np.array(bgs_matches) - np.array(sdss_matches)

# bin by sdss_matches
bins = np.linspace(0.8, 2.5, 30)
digitized = np.digitize(sdss_matches, bins)
bin_means = [difference[digitized == i].mean() for i in range(1, len(bins))]
#bin_stds = np.array([difference[digitized == i].std() for i in range(1, len(bins))])
bin_counts = [np.sum(digitized == i) for i in range(1, len(bins))]

# Calculate 1 sigma error bars
bin_intervals1 = [np.percentile(difference[digitized == i], [16, 84]) for i in range(1, len(bins))]
bin_lows1 = np.array([interval[0] for interval in bin_intervals1])
bin_highs1 = np.array([interval[1] for interval in bin_intervals1])

# Calculate 2 sigma error bars
bin_intervals = [np.percentile(difference[digitized == i], [2.5, 97.5]) for i in range(1, len(bins))]
bin_lows = np.array([interval[0] for interval in bin_intervals])
bin_highs = np.array([interval[1] for interval in bin_intervals])

# Convert yerr to a format that plt.errorbar can handle
yerr_1sigma = np.array([bin_means - bin_lows1, bin_highs1 - bin_means])
yerr_2sigma = np.array([bin_means - bin_lows, bin_highs - bin_means])

plt.errorbar(bins[1:], bin_means, yerr=yerr_1sigma, fmt='o', label='Mean Difference (1 sigma)', color='k')
plt.errorbar(bins[1:], bin_means, yerr=yerr_2sigma, fmt='o', label='Mean Difference (2 sigma)', alpha=0.5)

plt.xlabel('SDSS Dn4000')
plt.ylabel('BGS - SDSS Dn4000')
plt.legend()
plt.title("Shared Targets: Dn4000 Comparison")
#draw horizontal line at 0.0
plt.axhline(0.0, color='r', linestyle='--')

In [ ]:
# Compare quescient evaluation to SDSS for matched ones
#general_match = ang_distances < 1.0
bgs_quiescent_matched = quiescent[match_found_filter]
print(len(bgs_quiescent_matched))

bgs_matches = dn4000[match_found_filter]
sdss_indexes_q = neighbor_indexes[match_found_filter]
sdss_quiescent_matched = sdss.iloc[sdss_indexes_q]['QUIESCENT'].to_numpy()

# Percent that agree on quiescent classification
print(f"{np.sum(bgs_quiescent_matched == sdss_quiescent_matched) / len(bgs_quiescent_matched)} of the matched galaxies agree on quiescent classification using production methods")

bgs_quescient_via_g_r_only = is_quiescent_BGS_gmr(None, G_R_k_GAMA[match_found_filter])

print(f"{np.sum(bgs_quescient_via_g_r_only == sdss_quiescent_matched) / len(bgs_quescient_via_g_r_only)} of the matched galaxies agree on quiescent classification when only using BGS global g-r cut")


## Dn4000 Lgal Bin Analysis

Run Color Analysis and Dn4000 Comparison first

In [ ]:
# Make a plot of Dn4000 in each logLgal bin
# Show SDSS and my new Dn4000 model quiescent breakpoints
fig,axes=plt.subplots(dpi=80, figsize=(18, 8), ncols=4, nrows=2)
axes = np.ravel(axes)

for i in range(0, len(BGS_LOGLGAL_BINS)-1):
    galaxy_idx_for_this_bin = logLgal_bin_idx == i+1

    junk=axes[i].hist(dn4000[galaxy_idx_for_this_bin], bins=np.arange(1,2.2,0.02), label=f"Dn4000 for L Bin {i+1}", align='mid')
    axes[i].legend()
    axes[i].set_xlim(1.1, 2.1)
    axes[i].set_xticks(np.arange(1.2, 2.1, 0.1))
    axes[i].set_xlabel("DN4000")
    axes[i].set_ylabel("Count")

    # draw a vertical line at get_SDSS_Dcrit(logLgal)
    midpoint = (BGS_LOGLGAL_BINS[i] + BGS_LOGLGAL_BINS[i+1]) / 2
    axes[i].axvline(x=get_SDSS_Dcrit(midpoint), color='r', linestyle='-', alpha=1.0, linewidth=3)
    axes[i].axvline(x=get_ian_Dcrit(midpoint), color='k', linestyle='--', alpha=1.0, linewidth=3)


axes = np.reshape(axes, (2, len(BGS_LOGLGAL_BINS)//2))
plt.tight_layout()

In [ ]:
np.isnan(dn4000).sum()

In [ ]:
print(len(dn4000))
print(len(logLgal_bin_idx))

In [ ]:
from sklearn.mixture import GaussianMixture
import numpy as np
import matplotlib.pyplot as plt

# Function to fit a Gaussian Mixture Model and plot the results
def fit_gmm_and_plot(dn4000_data, logLgal_bin_idx, num_bins):
    def find_intersection(mean1, cov1, mean2, cov2):
        # Coefficients of the quadratic equation (a*x^2 + b*x + c = 0)
        a = 1 / cov1 - 1 / cov2
        b = -2 * (mean1 / cov1 - mean2 / cov2)
        c = (mean1**2 / cov1 - mean2**2 / cov2) - 2 * np.log(np.sqrt(cov2 / cov1))
        
        # Solve the quadratic equation
        roots = np.roots([a, b, c])
        return roots
    
    midpoints = []

    for i in range(1, num_bins + 1):
        galaxy_idx_for_this_bin = logLgal_bin_idx == i
        dn4000_bin = dn4000_data[galaxy_idx_for_this_bin]

        if len(dn4000_bin) == 0:
            continue

        # Fit a Gaussian Mixture Model with 2 components
        gmm = GaussianMixture(n_components=2, random_state=0)
        dn4000_bin_reshaped = dn4000_bin.reshape(-1, 1)
        gmm.fit(dn4000_bin_reshaped)

        # Get the parameters of the fitted Gaussians
        means = gmm.means_.flatten()
        covariances = gmm.covariances_.flatten()
        weights = gmm.weights_.flatten()

        # Find the intersection points
        intersections = find_intersection(means[0], covariances[0], means[1], covariances[1])
        midpoints.append(np.max(intersections))

        # Plot the histogram and the fitted Gaussians
        plt.figure(dpi=80, figsize=(14, 6))
        plt.hist(dn4000_bin, bins=np.arange(1, 2.2, 0.02), density=True, alpha=0.6, label=f"Dn4000 for logLgal Bin {i}")
        x = np.linspace(1, 2.2, 1000)
        y = (weights[0] * np.exp(-0.5 * ((x - means[0]) ** 2) / covariances[0]) / np.sqrt(2 * np.pi * covariances[0]) +
             weights[1] * np.exp(-0.5 * ((x - means[1]) ** 2) / covariances[1]) / np.sqrt(2 * np.pi * covariances[1]))
        plt.plot(x, y, label='Gaussian Mixture Model')
        for intersection in intersections:
            if 1 <= intersection <= 2.2:
                plt.axvline(intersection, color='r', linestyle='--', label=f'Intersection at {intersection:.2f}')
        plt.legend()
        plt.xlabel('Dn4000')
        plt.xticks(np.arange(1, 2.2, 0.05))
        plt.ylabel('Density')
        plt.title(f'Gaussian Mixture Model for logLgal Bin {i}')
        plt.show()

        print(f"Bin {i}:")
        print(f"Means: {means}")
        print(f"Covariances: {covariances}")
        print(f"Weights: {weights}")
        print(f"Intersections: {intersections}")

    return midpoints

# Number of logLgal bins
num_bins = len(BGS_LOGLGAL_BINS) - 1

# Fit the GMM and plot the results
good = ~np.isnan(dn4000)
midpoints = fit_gmm_and_plot(dn4000[good], logLgal_bin_idx[good], num_bins)


In [ ]:
print(np.array(BGS_LOGLGAL_BINS[1:]))
print(np.array(BGS_LOGLGAL_BINS[:-1]))


In [ ]:
# Assuming logLgal_bin_idx and log_L_gal are already defined
logLgal_bin_idx = np.digitize(log_L_gal, BGS_LOGLGAL_BINS)

# Calculate the mean value of LOGLGAL in each bin
mean_logLgal_per_bin = []
for i in range(1, len(BGS_LOGLGAL_BINS)):
    bin_values = log_L_gal[logLgal_bin_idx == i]
    mean_logLgal_per_bin.append(np.mean(bin_values))

# Print the mean values
for i, mean_value in enumerate(mean_logLgal_per_bin, 1):
    print(f"Mean LOGLGAL in bin {i}: {mean_value}")

In [ ]:
from scipy.special import erf
from scipy.optimize import curve_fit
import numpy as np

values = [1.41675126, 1.46407341, 1.51939392, 1.58815607,1.63857298, 1.67328342, 1.70584468, 1.74753757]

# Define the fitting function
def fitting_function(logLgal, A, B, C, D):
    return A + B*(1 + erf((logLgal - C) / D))

priors = [1.42, .175, 9.9, 0.8]


# Fit the function to the midpoints
popt, pcov = curve_fit(fitting_function, mean_logLgal_per_bin, values, p0=priors, maxfev=10000)

print(f"Fitted parameters: A={popt[0]:.3f}, B={popt[1]:.3f}, C={popt[2]:.3f}, D={popt[3]:.3f}")

# Plot the results
plt.figure(dpi=80, figsize=(8, 6))
plt.plot(mean_logLgal_per_bin, values, 'o', label='Midpoints')
x = np.linspace(8.0, 11.3, 1000)
y = fitting_function(x, *popt)
#plt.plot(x, y, label='Fitted Function')
plt.plot(x, get_SDSS_Dcrit(x), label='SDSS Dcrit')
plt.plot(x, get_ian_Dcrit(x), label='New BGS Dcrit')
plt.xlabel('logLgal')
plt.ylabel('Dn4000 Midpoint')
plt.legend()
plt.show()



In [ ]:
# Now create a fitting function for the results with parameters A,B,C,D
# A + (B) * (1 + special.erf((logLgal - C) / D))


# Randoms Analysis for Footprint

In [ ]:
build_sv3_clustering_randoms_files()
build_sv3_full_randoms_files()

In [ ]:
build_y3_likesv3_clustering_randoms_files()


In [ ]:
build_y1_randoms_files()
build_y3_randoms_files()

In [ ]:
sv3_randoms = pickle.load(open(MY_RANDOMS_SV3_MINI, 'rb'))
tiles_BGS = read_tiles_Y3_sv3()

In [ ]:
# Plot the randoms positions
fig=make_map(sv3_randoms['RA'].to_numpy(), sv3_randoms['DEC'].to_numpy())
#make_map(ra, dec, fig=fig, alpha=0.05)


# Some zoom-ins

# These are the two regions we cut due to poor Y3 overlap
#plot_positions(randoms_df0, randoms_df0[randoms_df0.NTILE_MINE >= 10], tiles_df=tiles_BGS, DEG_LONG=7, ra_min=192, dec_min=23, split=False)

plot_positions(sv3_randoms, sv3_randoms[sv3_randoms.NTILE_MINE >= 10], DEG_LONG=5, ra_min=213, dec_min=50, split=False)

In [ ]:
# CORRECT FOOTPRINTS
# These results copied into the groupcatalog.py file foorprints

def frac_sky_from_randoms(randoms: pd.DataFrame):
    """
    Provide a single randoms file at 2500 targets / square degrees for this, not a larger combined file.
    """
    RANDOMS_DENSITY = 2500 # per square degree, Ashley Ross paper on LSS pipeline or elsewhere in docs

    for i in range(1, 11):
        n_pass_filter = randoms.NTILE_MINE >= i
        n_pass_footprint = len(randoms[n_pass_filter].RA) / RANDOMS_DENSITY # in degrees squared
        n_pass_frac_area = n_pass_footprint / DEGREES_ON_SPHERE
        print(f"BGS {i}pass Footprint calculated from randoms is {n_pass_footprint} square degrees or frac_area={n_pass_frac_area}")

print("-=- SV3 18 regions -=-")
frac_sky_from_randoms(pickle.load(open(MY_RANDOMS_SV3_MINI, 'rb')))

print("-=- SV3 20 regions -=-")
frac_sky_from_randoms(pickle.load(open(MY_RANDOMS_SV3_MINI_20, 'rb')))

print("-=- Y1 -=-")
frac_sky_from_randoms(pickle.load(open(MY_RANDOMS_Y1_MINI, 'rb')))

print("-=- Y3 -=-")
frac_sky_from_randoms(pickle.load(open(MY_RANDOMS_Y3_MINI, 'rb')))